## Imports

In [13]:
import pandas as pd
import os

from src import files_import
from src import to_csv
from src import cleaning

## Variables

In [14]:
# Define the folder containing the files
EMA_dictionary = "data/StudentLife/dataset/EMA"
mood_directory_path = "data/StudentLife/dataset/EMA/response/Mood"
mood1_directory_path = "data/StudentLife/dataset/EMA/response/Mood 1"
mood2_directory_path = "data/StudentLife/dataset/EMA/response/Mood 2"
exercise_directory_path = "data/StudentLife/dataset/EMA/response/Exercise"
stress_directory_path = "data/StudentLife/dataset/EMA/response/Stress"
social_directory_path = "data/StudentLife/dataset/EMA/response/Social"
sleep_directory_path = "data/StudentLife/dataset/EMA/response/Sleep"
activity_directory_path = "data/StudentLife/dataset/EMA/response/Activity"


# Define single file paths
grades_file_path = "data/StudentLife/dataset/education/grades.csv"

# List of datasets and their directory paths
survey_datasets = {
    "Mood": {'Path' : mood_directory_path, 'Important_Column' : 'happyornot'},
    "Mood1":  {'Path' : mood1_directory_path, 'Important_Column' : 'tomorrow'},
    "Mood2":  {'Path' : mood2_directory_path, 'Important_Column' : 'how'},
    "Exercise":  {'Path' : exercise_directory_path, 'Important_Column' : 'have'},
    "Stress":  {'Path' : stress_directory_path, 'Important_Column' : 'level'},
    "Social":  {'Path' : social_directory_path, 'Important_Column' : 'number'},
    "Sleep":  {'Path' : sleep_directory_path, 'Important_Column' : 'hour'},
    "Activity":  {'Path' : activity_directory_path, 'Important_Column' : 'working'},
}




## Download the data

In [ ]:
all_data = {}
for survey, dictionary in survey_datasets.items():
    combined_data = to_csv.folder_to_csv(dictionary['Path'], dictionary['Important_Column'], 0)
    all_data[survey] = combined_data


## Analysing

### Making a combined csv with all types of data

In [24]:
combined_csv = {}
# Transformation dictionaries
social_mapping = {
    1:2, 2:7, 3:15, 4:30, 5:75, 6:150
}

social_data= all_data['Social']
all_data['Social']['number_of_people'] = social_data['number'].map(social_mapping)



average_social = all_data['Social'].groupby('User')['number_of_people'].mean()
average_stress = all_data['Stress'].groupby('User')['level'].mean()
combined_csv['number_of_people'] = average_social
combined_csv['avg_stress_level'] = average_social

df = pd.DataFrame.from_dict(combined_csv, orient='columns')
df.to_csv("output.csv", index=True)
